In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9fz8wyt_
  Running command git clone --filter=blob:none -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9fz8wyt_
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from sklearn.model_selection import train_test_split
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
filepath_train = os.getenv('HOME')+'/aiffel/dktc/data/train.csv'

train = pd.read_csv(filepath_train)

train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [4]:
len(train)

3950

In [5]:
train.isnull().sum()

idx             0
class           0
conversation    0
dtype: int64

In [6]:
train['class'].drop_duplicates()

0          협박 대화
2      기타 괴롭힘 대화
3          갈취 대화
5    직장 내 괴롭힘 대화
Name: class, dtype: object

In [7]:
# 텍스트 클렌징 함수
def clean_text(text):
    # 불필요한 특수 문자, 숫자 제거 (한글, 영문, 공백 제외)
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\s]", "", text)
    text = re.sub(r"\n", " ", text)
    return text

train['conversation'] = train['conversation'].apply(clean_text)

In [8]:
# 클렌징 후 비어 있는 대화가 있는지 확인
empty_conversations = train[train['conversation'].str.strip() == '']
print(f"Empty conversations: {len(empty_conversations)}")

Empty conversations: 0


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /aiffel/aiffel/dktc/.cache/kobert_v1.zip


In [ ]:
train.loc[(train['class'] == "협박 대화"), 'label'] = 0  # 협박 대화 => 0
train.loc[(train['class'] == "갈취 대화"), 'label'] = 1  # 갈취 대화 => 1
train.loc[(train['class'] == "직장 내 괴롭힘 대화"), 'label'] = 2  # 직장 내 괴롭힘 대화 => 2
train.loc[(train['class'] == "기타 괴롭힘 대화"), 'label'] = 3  # 기타 괴롭힘 대화 => 3
train.loc[(train['class'] == "일반 대화"), 'label'] = 4  # 일반 대화 => 4

data_list = []
for content, label in zip(train['conversation'], train['label'])  :
    temp = []
    temp.append(content)
    temp.append(str(int(label)))

    data_list.append(temp)

In [ ]:
train.drop(columns = 'idx', inplace = True)

In [ ]:
data_list[0]

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, random_state = 42)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        
        # 레이블이 없는 경우를 처리
        if label_idx is not None:
            self.labels = [np.int32(i[label_idx]) for i in dataset]
        else:
            self.labels = None

    def __getitem__(self, i):
        if self.labels is not None:
            return self.sentences[i] + (self.labels[i], )
        else:
            return self.sentences[i]

    def __len__(self):
        return len(self.sentences)

In [ ]:
## Setting parameters
max_len = 130 # 이것도 변경되는 것이 좋을 것 같다
batch_size = 32 # 고정! 이걸 넘기면 모델이 안돌아감
warmup_ratio = 0.2 #이걸 수정해보자
num_epochs = 5 
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
# 배치 및 데이터로더 설정 (num_workers 최적화)
# num_workers는 시스템에 맞춰 적절한 값을 설정 (예: CPU 코어 수에 맞춰 4나 8 정도로 설정)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4, shuffle=False)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=5,
                 dr_rate=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, token_ids, segment_ids):
        # attention mask는 token_ids에서 패딩(0)이 아닌 부분을 1로 설정
        attention_mask = (token_ids != 0).float()
        
        # BERT 모델에서 pooler 출력 사용
        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
        
        # 드롭아웃 적용
        if self.dr_rate:
            out = self.dropout(pooler)
        
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().item() / max_indices.size(0)
    return train_acc

def calc_metrics(preds, Y):
    # preds는 detach하여 그래디언트를 추적하지 않도록 하고, CPU로 옮긴 후 numpy 배열로 변환
    pred_labels = preds.detach().cpu().numpy()
    true_labels = Y.cpu().numpy()
    
    # 정확도 계산: pred_labels == true_labels 결과를 명시적으로 int로 변환 후 합산
    accuracy = (pred_labels == true_labels).astype(int).sum() / len(true_labels)
    
    # F1, Precision, Recall 계산, zero_division 옵션 추가로 경고 억제
    f1 = f1_score(true_labels, pred_labels, average='weighted', zero_division=0)
    precision = precision_score(true_labels, pred_labels, average='weighted', zero_division=0)
    recall = recall_score(true_labels, pred_labels, average='weighted', zero_division=0)
    
    return accuracy, f1, precision, recall

In [ ]:
'''
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args

# 하이퍼파라미터 공간 정의
space = [
    Integer(60, 220, name='max_len'),  # max_len: 60~220, 10씩 증가
    Real(0.0, 0.4, name='warmup_ratio'),  # warmup_ratio: 0~0.4
    Integer(3, 10, name='num_epochs'),  # num_epochs: 3~10
    Real(1e-5, 1e-4, prior='log-uniform', name='learning_rate'),  # learning_rate: 1e-5~1e-4
    Real(0.1, 0.5, name='dr_rate')  # dr_rate: 0.1~0.5
]

@use_named_args(space)
def objective(max_len, warmup_ratio, num_epochs, learning_rate, dr_rate):
    # 모델 설정
    model = BERTClassifier(bertmodel, dr_rate=dr_rate).to(device)

    # 옵티마이저와 스케줄러 설정
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    t_total = len(train_dataloader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

    # 학습 및 성능 평가
    for e in range(num_epochs):
        train_acc = 0.0
        train_f1, train_precision, train_recall = 0.0, 0.0, 0.0
        test_acc, val_f1, val_precision, val_recall = 0.0, 0.0, 0.0, 0.0

        model.train()
        for batch_id, (token_ids, _, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
            optimizer.zero_grad()
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            label = label.long().to(device)
            
            out = model(token_ids, segment_ids)
            loss = loss_fn(out, label)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()

            batch_acc, batch_f1, batch_precision, batch_recall = calc_metrics(out, label)
            train_acc += batch_acc
            train_f1 += batch_f1
            train_precision += batch_precision
            train_recall += batch_recall

        model.eval()
        with torch.no_grad():
            for batch_id, (token_ids, _, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
                token_ids = token_ids.long().to(device)
                segment_ids = segment_ids.long().to(device)
                label = label.long().to(device)

                out = model(token_ids, segment_ids)
                batch_acc, batch_f1, batch_precision, batch_recall = calc_metrics(out, label)
                test_acc += batch_acc
                val_f1 += batch_f1
                val_precision += batch_precision
                val_recall += batch_recall

    # 최종 검증 F1-score 반환
    return -val_f1 / len(test_dataloader)  # 최적화 방향을 위해 음수로 반환

# 베이지안 최적화 실행
result = gp_minimize(objective, space, n_calls=50, random_state=42)

# 최적의 하이퍼파라미터 출력
print("Best hyperparameters: ", result.x)
'''

In [ ]:
# 확률 기반 이상치 탐지 함수
def classify_with_outlier_detection(logits, threshold=0.95):
    probs = F.softmax(logits, dim=1)  # 클래스별 확률 계산
    max_probs, preds = torch.max(probs, dim=1)  # 가장 높은 확률과 그 클래스
    
    # 확률이 threshold 이하인 경우 '일반 대화'로 분류 (label: 4)
    preds[max_probs < threshold] = 4
    return preds

In [25]:
# 수동으로 최적의 하이퍼파라미터 설정
max_len = 69
warmup_ratio = 0.28879950890672995
num_epochs = 10
learning_rate = 1.0017947833154703e-05
dr_rate = 0.4968846237164871

# 데이터셋 및 데이터로더 생성
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4, shuffle=False)

# 모델 생성
model = BERTClassifier(bertmodel, dr_rate=dr_rate).to(device)

# 옵티마이저와 스케줄러 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 최종 모델 학습
for epoch in range(num_epochs):
    model.train()
    train_acc, train_f1, train_precision, train_recall = 0.0, 0.0, 0.0, 0.0
    for batch_id, (token_ids, _, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        label = label.long().to(device)

        out = model(token_ids, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        # 확률 기반 이상치 탐지 적용 (임계값 threshold = 0.95)
        preds = classify_with_outlier_detection(out, threshold=0.95)

        # 성능 지표 계산: preds 사용
        batch_acc, batch_f1, batch_precision, batch_recall = calc_metrics(preds, label)
        train_acc += batch_acc
        train_f1 += batch_f1
        train_precision += batch_precision
        train_recall += batch_recall

    print(f'Epoch {epoch + 1}: Train Accuracy: {train_acc/len(train_dataloader)}, F1: {train_f1/len(train_dataloader)}, Precision: {train_precision/len(train_dataloader)}, Recall: {train_recall/len(train_dataloader)}')

# 테스트 데이터 예측 및 이상치 탐지 적용
model.eval()
test_acc, test_f1, test_precision, test_recall = 0.0, 0.0, 0.0, 0.0
with torch.no_grad():
    for batch_id, (token_ids, _, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        label = label.long().to(device)

        # 모델 예측
        out = model(token_ids, segment_ids)

        # 확률 기반 이상치 탐지 적용 (임계값 threshold = 0.95)
        preds = classify_with_outlier_detection(out, threshold=0.95)

        # 성능 지표 계산: preds 사용
        batch_acc, batch_f1, batch_precision, batch_recall = calc_metrics(preds, label)
        test_acc += batch_acc
        test_f1 += batch_f1
        test_precision += batch_precision
        test_recall += batch_recall

print(f'Test Accuracy: {test_acc/len(test_dataloader)}, F1: {test_f1/len(test_dataloader)}, Precision: {test_precision/len(test_dataloader)}, Recall: {test_recall/len(test_dataloader)}')


# 확률 기반 이상치 탐지 함수
def classify_with_outlier_detection(logits, threshold=0.95):
    probs = F.softmax(logits, dim=1)  # 클래스별 확률 계산
    max_probs, preds = torch.max(probs, dim=1)  # 가장 높은 확률과 그 클래스
    
    # 확률이 threshold 이하인 경우 '일반 대화'로 분류 (label: 4)
    preds[max_probs < threshold] = 4
    return preds


Epoch 2: Train Accuracy: 0.0, F1: 0.0, Precision: 0.0, Recall: 0.0
Epoch 3: Train Accuracy: 0.06902356902356901, F1: 0.09681959514620006, Precision: 0.20309869528619529, Recall: 0.06902356902356901
Epoch 4: Train Accuracy: 0.31502525252525254, F1: 0.3884927243852999, Precision: 0.6665374547240077, Recall: 0.31502525252525254
Epoch 5: Train Accuracy: 0.5913299663299663, F1: 0.6970007831120921, Precision: 0.9683058488929702, Recall: 0.5913299663299663
Epoch 6: Train Accuracy: 0.8499579124579124, F1: 0.9012767662346678, Precision: 0.9867901485088986, Recall: 0.8499579124579124
Epoch 7: Train Accuracy: 0.9452861952861953, F1: 0.965858582621741, Precision: 0.9922639317052199, Recall: 0.9452861952861953
Epoch 8: Train Accuracy: 0.9747474747474747, F1: 0.9840011113600675, Precision: 0.9957799644589416, Recall: 0.9747474747474747
Epoch 9: Train Accuracy: 0.9863215488215488, F1: 0.9909802409634184, Precision: 0.9969053244384684, Recall: 0.9863215488215488
Epoch 10: Train Accuracy: 0.99273989898

In [26]:
print(preds.shape)

torch.Size([22])


---

In [27]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

# 확률 기반 이상치 탐지 함수
def classify_with_outlier_detection(logits, threshold=0.95):
    probs = F.softmax(logits, dim=1)  # 클래스별 확률 계산
    max_probs, preds = torch.max(probs, dim=1)  # 가장 높은 확률과 그 클래스
    
    # 확률이 threshold 이하인 경우 '일반 대화'로 분류 (label: 4)
    preds[max_probs < threshold] = 4
    return preds

# test.json 파일 경로 설정
filepath_test = os.getenv('HOME')+'/aiffel/dktc/data/test.json'

# test.json 파일을 DataFrame으로 읽기 (pandas를 사용하는 경우)
test_data = pd.read_json(filepath_test)

# 각 열에 있는 'text' 데이터를 추출하여 하나의 리스트로 만듭니다.
test_list = []

for column in test_data.columns:
    conversation = test_data[column]['text']
    test_list.append([conversation])  # 각 대화를 리스트에 추가

# 텍스트 클렌징 함수 (train에서 사용한 것과 동일하게 적용)
def clean_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\s]", "", text)
    text = re.sub(r"\n", " ", text)
    return text

# 전처리 적용
test_list = [[clean_text(conversation[0])] for conversation in test_list]

# 토큰화 및 데이터셋 변환 (BERTDataset 클래스 및 토크나이저는 이미 정의되어 있다고 가정)
data_test = BERTDataset(test_list, 0, None, tok, max_len, True, False)

# 테스트 데이터 로더 생성
test_dataloader = DataLoader(data_test, batch_size=batch_size, num_workers=4)

# 모델 예측
model.eval()
predictions = []

with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        # 모델 예측
        out = model(token_ids, segment_ids)
        
        # 확률 기반 이상치 탐지 적용 (임계값 threshold = 0.95)
        preds = classify_with_outlier_detection(out, threshold=0.95)  # 확률 기반 분류 적용
        predictions.extend(preds.cpu().numpy())  # 예측된 클래스 저장

# 숫자 레이블을 클래스명으로 변환하는 매핑
label_mapping = {
    0: "0",
    1: "1",
    2: "2",
    3: "3",
    4: "4"  # '일반 대화' 추가
}

# 예측된 숫자 레이블을 클래스명으로 변환
predicted_classes = [label_mapping[pred] for pred in predictions]

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'idx': test_data.columns,  # 각 대화에 해당하는 열 이름 (예: 't_000', 't_001' 등)
    'target': predicted_classes  # 예측된 클래스명
})

# 제출 파일로 저장
submission.to_csv('./threshold095.csv', index=False)
print("Results saved to threshold095.csv")

/tmp/ipykernel_10426/4250175812.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/16 [00:00<?, ?it/s]

Results saved to threshold095.csv


In [28]:
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True